In [1]:
import numpy as np
import pandas as pd
import math
import requests 
import matplotlib as plt
import seaborn as sns
import yfinance as yf
import pandas_datareader as web
from pandas_datareader import data
from bs4 import BeautifulSoup as bs
from scipy import stats

In [2]:
tick = pd.read_csv("all_stocks")

In [15]:
final = pd.DataFrame(columns=[
                            'Ticker',
                            'Price',
                            '1y Price Return',
                            '1y percentile', 
                            '6m Price Return', 
                            '6m percentile', 
                            '3m Price Return', 
                            '3m percentile',
                            '1m Price Return', 
                            '1m percentile'
                            ])
for i in range(len(tick["0"])):
    print("Producing data for: ", tick["0"][i])
    try:
        d = yf.Ticker(tick["0"][i]).history(period='max')
        y = yf.Ticker(tick["0"][i]).history(period='1y')
        m6 = yf.Ticker(tick["0"][i]).history(period='6mo')
        m3 = yf.Ticker(tick["0"][i]).history(period='3mo')
        m1 = yf.Ticker(tick["0"][i]).history(period='1mo')
    except IndexError: 
        print("Couldn't find: ",tick["0"][i])
        continue
    except KeyError:
        continue
    
    d.reset_index(inplace=True)
    y.reset_index(inplace=True)
    m6.reset_index(inplace=True)
    m3.reset_index(inplace=True)
    m1.reset_index(inplace=True)

    d['Ticker'] = tick['0'][i]    
    try: 
        perc_change1y = ((y.iloc[-1]['Close'] - y.iloc[0]['Close'])/y.iloc[0]["Close"])
        d['1y Price Return'] = perc_change1y
        perc_change6m = ((m6.iloc[-1]['Close'] - m6.iloc[0]['Close'])/m6.iloc[0]["Close"])
        d['6m Price Return'] = perc_change6m
        perc_change3m = ((m3.iloc[-1]['Close'] - m3.iloc[0]['Close'])/m3.iloc[0]["Close"])
        d['3m Price Return'] = perc_change3m
        perc_change1m = ((m1.iloc[-1]['Close'] - m1.iloc[0]['Close'])/m1.iloc[0]["Close"])
        d['1m Price Return'] = perc_change1m

        d['Price'] = d['Close'] 
        d['1y percentile'] = 0
        d['6m percentile'] = 0
        d['3m percentile'] = 0
        d['1m percentile'] = 0
        d = d.iloc[-1]
    except IndexError: 
        continue
    print(d)

    #Change to keep only last 2 days!!! get date
    
    #Append latest value
    final = final.append(
        pd.Series(d,
        index=['Ticker',
                            'Price',
                            '1y Price Return',
                            '1y percentile', 
                            '6m Price Return', 
                            '6m percentile', 
                            '3m Price Return', 
                            '3m percentile',
                            '1m Price Return', 
                            '1m percentile']), 
        ignore_index = True
    )  

Producing data for:  AOS
Date               2022-12-23 00:00:00-05:00
Open                               56.049999
High                               56.939999
Low                                    55.84
Close                              56.900002
Volume                                879500
Dividends                                0.0
Stock Splits                             0.0
Ticker                                   AOS
1y Price Return                     -0.31561
6m Price Return                     0.030038
3m Price Return                     0.158284
1m Price Return                    -0.080181
Price                              56.900002
1y percentile                              0
6m percentile                              0
3m percentile                              0
1m percentile                              0
Name: 9890, dtype: object


<ipython-input-15-46534c27f6d6>:57: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final = final.append(


In [ ]:
final.to_csv("momentum_advanced_sheet")